In [6]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
dataset=pd.read_csv('creditcard.csv')
sc=StandardScaler()
dataset['Time']=sc.fit_transform(dataset['Time'].values.reshape(-1,1))
dataset['Amount']=sc.fit_transform(dataset['Amount'].values.reshape(-1,1))

In [15]:
data=dataset.iloc[:,:-1].values
labels=dataset['Class'].values
# training 
train_data,test_data,train_labels,test_labels=train_test_split(data,labels,test_size=0.2,random_state=2021)

In [16]:
#normalization 
min_val=tf.reduce_min(train_data)
max_val=tf.reduce_max(train_data)
train_data=(train_data-min_val) /(max_val - min_val)
test_data=(test_data - min_val) / (max_val - min_val)

In [17]:
#autoencoder
  
input_dim = train_data.shape[1] # Number of features in input data
 
encoding_dim =14  # Number of units in first encoder layer 
input_layer=tf.keras.layers.Input(shape=(input_dim))
encoder=tf.keras.layers.Dense(encoding_dim,activation='tanh')(input_layer)
encoder=tf.keras.layers.Dense(7,activation='relu')(encoder)
decoder=tf.keras.layers.Dense(encoding_dim,activation='relu')(encoder)
decoder=tf.keras.layers.Dense(input_dim,activation='tanh')(decoder)

autoencoder=tf.keras.Model(inputs=input_layer,outputs=decoder)

In [18]:
autoencoder.compile(optimizer='adam',loss='mean_squared_error')


In [22]:
history= autoencoder.fit(train_data,train_data,epochs=50,batch_size=64,shuffle=True,validation_data=(test_data,test_data))

Epoch 1/50


2024-11-06 16:29:24.246870: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3561/3561 [==============================] - 2s 456us/step - loss: 0.0053 - val_loss: 2.0565e-05
Epoch 2/50
3561/3561 [==============================] - 2s 442us/step - loss: 2.0620e-05 - val_loss: 2.0881e-05
Epoch 3/50
3561/3561 [==============================] - 2s 443us/step - loss: 2.0449e-05 - val_loss: 2.0207e-05
Epoch 4/50
3561/3561 [==============================] - 2s 438us/step - loss: 2.0091e-05 - val_loss: 1.9868e-05
Epoch 5/50
3561/3561 [==============================] - 2s 439us/step - loss: 1.9688e-05 - val_loss: 1.9371e-05
Epoch 6/50
3561/3561 [==============================] - 2s 440us/step - loss: 1.9301e-05 - val_loss: 1.8751e-05
Epoch 7/50
3561/3561 [==============================] - 2s 441us/step - loss: 1.8882e-05 - val_loss: 1.9506e-05
Epoch 8/50
3561/3561 [==============================] - 2s 443us/step - loss: 1.8520e-05 - val_loss: 1.8578e-05
Epoch 9/50
3561/3561 [==============================] - 2s 441us/step - loss: 1.8248e-05 - val_loss: 1.9089e-05
Epoch 1

In [25]:
test_x_predictions=autoencoder.predict(test_data)
mse=np.mean(np.power(test_data - test_x_predictions,2),axis=1)
threshold_fixed=50
pred_y=[1 if e > threshold_fixed else 0 for e in mse]

1781/1781 [==============================] - 0s 234us/step


In [27]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_labels,pred_y))
      

[[56859     0]
 [  103     0]]
